In [1]:
import pandas as pd
import numpy as np
import os

import time
from alibi_detect.od import SpectralResidual

In [2]:
def get_data():
    data_path = '../../../../data/train_data/host'
    dfs = {}
    for file in os.listdir(data_path):
        print('Reading ' + file)
        dfs[file[:-4]] = pd.read_csv(data_path+'/'+file) 
    return dfs

def load_tiago_test(test):
    data_path = '../../../../data/tiago_tests/test'
    df = pd.read_csv(data_path+str(test)+'/kpis.csv')
    return df

def load_test():
    data_path = '../../../../data/test_data/host'
    df = pd.DataFrame(['item_id','name','bomc_id','timestamp','value','cmdb_id'])
    for file in os.listdir(data_path):
        df = pd.concatenate(df, pd.read_csv(data_path+'/'+file)) 
    return dfs

In [3]:
def get_past(df, curr_time, window_size):
    step_size = 1000*60
    df_new = pd.DataFrame(columns=['timestamp', 'value'])

    for time in [curr_time - step_size*x for x in range(window_size)][::-1]:
        if len(df[df.timestamp == time]) == 0:
            plug_time = np.max(df[df.timestamp < time]['timestamp'])
            row = {}
            row['timestamp'] = time
            if len(df.loc[df.timestamp==plug_time, 'value']) == 0:
                row['value'] = 0
            else: 
                row['value'] = df.loc[df.timestamp==plug_time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
        else:
            row = {}
            row['timestamp'] = time
            row['value'] = df.loc[df.timestamp==time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
    return df_new

In [26]:
def get_key_thresh(host):
    df_thresh = pd.read_csv('thresh.data')
    df_thresh = df_thresh[df_thresh.host==host]

    thresh = df_thresh[['name', 'thresh']].set_index('name')
    thresh = thresh.to_dict()['thresh']
    return thresh

def find_anoms(hosts, df):
    start = time.time()
    anoms = []
    for host in hosts:
        start_host = time.time()
        print('*'*60)
        print('Checking ', host)
        thresholds = get_key_thresh(host)
        for kpi in thresholds:
            thresh = thresholds[kpi]
            print('Threshold ', thresh, kpi)
            od = SpectralResidual(
                threshold=thresh,
                window_amp=30,
                window_local=30,
                n_est_points=5,
                n_grad_points=5
            )
            df_hk = df[df.cmdb_id == host][df.name == kpi]
            
            if len(df_hk) == 0:
                print('No valid data to use, skipping')
                continue
            
            if kpi == 'container_cpu_used':
                print(df_hk.timestamp.apply(lambda x: np.datetime64(x, 'ms')), df_hk.value)
            data = get_past(df_hk, np.max(df_hk.timestamp.unique()), 30)['value']
            data = np.concatenate([data, data])
            
            if thresh == -1:
                if data[-1] > 0:
                    print("Non-Zero Anomaly!")
                    anoms.append((host, kpi))
            elif np.isnan(thresh):
                print('NaN, Skipping!')
            else:
                outliers = od.predict(data)['data']
                print(outliers['instance_score'])
                if np.mean(data) == 0 or np.std(data) == 0:
                    print("Mean is zero or std is zero")
                    anoms.append((host,kpi))
                if np.sum(outliers['is_outlier'][-31:-30]) > 0:
                    print(outliers['is_outlier'])
                    print("ST Threshold Anomaly!")
                    anoms.append((host,kpi))
        print(host, ' completed in ', time.time() - start_host)
    print('Completed detection of ', len(hosts), 'hosts in ', time.time() - start, 'seconds')
    return anoms


find_anoms(['docker_008'], load_tiago_test(9))

************************************************************
Checking  docker_008
Threshold  -1.0 container_session_used
Threshold  -1.0 container_fgct
Threshold  3.2055302236081182 container_cpu_used
28919   2020-04-10 19:40:26
28947   2020-04-10 19:45:26
28948   2020-04-10 19:44:26
28949   2020-04-10 19:43:26
28950   2020-04-10 19:42:25
28951   2020-04-10 19:41:25
29298   2020-04-10 19:50:27
29299   2020-04-10 19:49:24
29300   2020-04-10 19:48:24
29301   2020-04-10 19:47:26
29302   2020-04-10 19:46:26
29738   2020-04-10 19:55:26
29739   2020-04-10 19:54:26
29740   2020-04-10 19:53:24
29741   2020-04-10 19:52:25
29742   2020-04-10 19:51:24
30015   2020-04-10 19:59:26
30016   2020-04-10 19:58:25
30017   2020-04-10 19:57:25
30018   2020-04-10 19:56:25
30312   2020-04-10 20:00:26
30378   2020-04-10 20:05:26
30379   2020-04-10 20:04:36
30380   2020-04-10 20:03:24
30381   2020-04-10 20:02:25
30382   2020-04-10 20:01:25
30738   2020-04-10 20:10:25
30739   2020-04-10 20:09:24
30740   2020-04

<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:96: RuntimeWarning: divide by zero encountered in log
  log_amp = np.log(amp)
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:99: RuntimeWarning: invalid value encountered in subtract
  res_amp = log_amp - ma_log_amp
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:207: RuntimeWarning: invalid value encountered in greater
  outlier_pred = (iscore > self.

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
Mean is zero or std is zero
Threshold  -1.0 container_thread_idle
Threshold  -1.0 container_thread_running
Threshold  -1.0 container_thread_total
Threshold 

<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


 -1.0 container_fgc
Threshold  -1.0 container_thread_used_pct
Threshold  0.8409867960331752 container_mem_used
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
Mean is zero or std is zero
docker_008  completed in  0.6365790367126465
Completed detection of  1 hosts in  0.6366016864776611 seconds


<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-26-197b7060cd98>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:96: RuntimeWarning: divide by zero encountered in log
  log_amp = np.log(amp)
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:99: RuntimeWarning: invalid value encountered in subtract
  res_amp = log_amp - ma_log_amp
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:207: RuntimeWarning: invalid value encountered in greater
  outlier_pred = (iscore > self.threshold).astype(int)


[('docker_008', 'container_cpu_used'), ('docker_008', 'container_mem_used')]

In [ ]:
import seaborn as sns

df = load_test(2)
df = df[df.cmdb_id=='os_018'][df.name=='Buffers_used']
sns.lineplot(x='timestamp', y='value',data=df)